# Capstone project

## Opening a Hotel in Toronto

### Obtaining all the data from by scraping data and putting it in a csv
### Obtain the venue data for the neighborhoods from Foursquare API
### Explore and cluster the neighborhoods
### Select the best cluster to open a new shopping mall

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0



geopy-2.1.0          | 64 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

# Downloading all the needed information and putting it into a dataframe

In [2]:
df = pd.read_csv('Toronto.csv')

# Exploring the Data

In [3]:
df

,Neighbourhood,Latitude,Longitude
0,Parkwoods,43.753259,-79.329656
1,Victoria Village,43.725882,-79.315572
2,"Regent Park, Harbourfront",43.654260,-79.360636
3,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,"Malvern, Rouge",43.806686,-79.194353
7,Don Mills,43.745906,-79.352188
8,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,"Garden District, Ryerson",43.657162,-79.378937


# Let's see all the neighbourhoods in Toronto

In [4]:
df['Neighbourhood'].unique()

array(['Parkwoods', 'Victoria Village', 'Regent Park, Harbourfront',
       'Lawrence Manor, Lawrence Heights',
       "Queen's Park, Ontario Provincial Government",
       'Islington Avenue, Humber Valley Village', 'Malvern, Rouge',
       'Don Mills', 'Parkview Hill, Woodbine Gardens',
       'Garden District, Ryerson', 'Glencairn',
       'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
       'Rouge Hill, Port Union, Highland Creek', 'Woodbine Heights',
       'St. James Town', 'Humewood-Cedarvale',
       'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood',
       'Guildwood, Morningside, West Hill', 'The Beaches', 'Berczy Park',
       'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street',
       'Christie', 'Cedarbrae', 'Hillcrest Village',
       'Bathurst Manor, Wilson Heights, Downsview North',
       'Thorncliffe Park', 'Richmond, Adelaide, King',
       'Dufferin, Dovercourt Village', 'Scarborough Village',
       'Fairview, H

# Let's visualise all neughbourhoods in Toronto

In [5]:
# create map of Toronto using latitude and longitude values
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Defining Foursquare Credentials and Version

In [6]:
CLIENT_ID = 'RT5PLOZFD0HDEN2YCY1O3YBSR050VXP1MIRES0Y3QUNU3P0Y' # your Foursquare ID
CLIENT_SECRET = 'MY0UUKHAZU34WTZT1JSPB5LWTGFTT03UT4KGSIM0XGNE2TTD' # your Foursquare Secret
VERSION = '20210101' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RT5PLOZFD0HDEN2YCY1O3YBSR050VXP1MIRES0Y3QUNU3P0Y
CLIENT_SECRET:MY0UUKHAZU34WTZT1JSPB5LWTGFTT03UT4KGSIM0XGNE2TTD


## Now, let's get the top 100 venues that are in Toronto within a radius of 2000 meters. First, let's create the GET request URL

In [7]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [8]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(8590, 7)


,Neighbourhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
2,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
3,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
4,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518,Supermarket


## Let's check how many venues were returned for each neighborhood

In [9]:
venues_df.groupby(["Neighbourhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhood,,,,,,
Agincourt,100,100,100,100,100,100
"Alderwood, Long Branch",100,100,100,100,100,100
"Bathurst Manor, Wilson Heights, Downsview North",54,54,54,54,54,54
Bayview Village,53,53,53,53,53,53
"Bedford Park, Lawrence Manor East",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",43,43,43,43,43,43
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100


## Let's find out how many unique categories can be curated from all the returned venues

In [10]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 330 uniques categories.


In [11]:
venues_df['VenueCategory'].unique()[:50]

array(['Caribbean Restaurant', 'Café', 'Golf Course', 'Liquor Store',
       'Supermarket', 'Event Space', 'Middle Eastern Restaurant', 'Park',
       'Seafood Restaurant', 'Mediterranean Restaurant',
       'Gym / Fitness Center', 'Gym', 'Grocery Store',
       'Italian Restaurant', 'Discount Store', 'Pool Hall',
       'Fried Chicken Joint', 'Steakhouse', 'Japanese Restaurant',
       'Ice Cream Shop', 'American Restaurant', 'Coffee Shop',
       'Burger Joint', 'Fish & Chips Shop', 'Pharmacy',
       'Persian Restaurant', 'Pizza Place', 'Fast Food Restaurant',
       'Chinese Restaurant', 'Salad Place', 'Filipino Restaurant',
       'Paper / Office Supplies Store', 'Beer Store', 'Thai Restaurant',
       'Hotel', 'Sandwich Place', 'Gas Station', 'Bank', 'Breakfast Spot',
       'Hakka Restaurant', 'Bar', 'Asian Restaurant', 'Wings Joint',
       'Automotive Shop', 'Bus Stop', 'Furniture / Home Store',
       'Cosmetics Shop', 'Diner', 'Greek Restaurant', 'Sushi Restaurant'],
      d

## Let's see if Hotel is in the returned category

In [72]:
# check if the results contain "Hotel"
"Hotel" in venues_df['VenueCategory'].unique()

True

In [59]:
# one hot encoding
t_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
t_onehot['Neighbourhoods'] = venues_df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [t_onehot.columns[-1]] + list(t_onehot.columns[:-1])
t_onehot = t_onehot[fixed_columns]

print(t_onehot.shape)
t_onehot.head()

(8590, 331)


,Neighbourhoods,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki 

In [60]:
t_grouped = t_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(t_grouped.shape)
t_grouped

(99, 331)


,Neighbourhoods,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki 

# Let's see number of hotels present

In [61]:
len(t_grouped[t_grouped["Hotel"] > 0])

47

## Let's see the number of hotels in all Neighbourhood

In [62]:
t_sh = t_grouped[["Neighbourhoods","Hotel"]]
t_sh

,Neighbourhoods,Hotel
0,Agincourt,0.000000
1,"Alderwood, Long Branch",0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000
3,Bayview Village,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000
5,Berczy Park,0.060000
6,"Birch Cliff, Cliffside West",0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.010000
8,"Business reply mail Processing Centre, South C...",0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.030000


# Cluster Neighborhoods

## Run k-means to cluster the neighborhood into 3 clusters.

In [63]:
# set number of clusters
kclusters = 3

t_clustering = t_sh.drop(["Neighbourhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 2, 1, 1, 1, 0], dtype=int32)

In [64]:
t_merged = t_sh.copy()

# add clustering labels
t_merged["Cluster Labels"] = kmeans.labels_

In [65]:
t_merged.rename(columns={"Neighbourhoods": "Neighbourhood"}, inplace=True)
t_merged.head()

,Neighbourhood,Hotel,Cluster Labels
0,Agincourt,0.0,1
1,"Alderwood, Long Branch",0.0,1
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,1
3,Bayview Village,0.0,1
4,"Bedford Park, Lawrence Manor East",0.0,1


## Let's see the ratio of hotels in each cluster

In [66]:
t_merged = t_merged.join(df.set_index("Neighbourhood"), on="Neighbourhood")
print(t_merged.shape)
t_merged.sort_values(["Cluster Labels"], inplace=True)
t_merged

(103, 5)


,Neighbourhood,Hotel,Cluster Labels,Latitude,Longitude
68,"Rouge Hill, Port Union, Highland Creek",0.025641,0,43.784535,-79.160497
15,Church and Wellesley,0.020000,0,43.665860,-79.383160
78,"Summerhill West, Rathnelly, South Hill, Forest...",0.030000,0,43.686412,-79.400049
13,Central Bay Street,0.030000,0,43.657952,-79.387383
31,"Garden District, Ryerson",0.030000,0,43.657162,-79.378937
39,"Humberlea, Emery",0.021277,0,43.724766,-79.532242
88,"West Deane Park, Princess Gardens, Martin Grov...",0.020833,0,43.650943,-79.554724
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.030000,0,43.628947,-79.394420
64,"Regent Park, Harbourfront",0.020000,0,43.654260,-79.360636
93,"Willowdale, Willowdale East",0.024390,0,43.770120,-79.408493


# Finally, let's visualize the resulting clusters

In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(t_merged['Latitude'], t_merged['Longitude'], t_merged['Neighbourhood'], t_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

# CLUSTER 1

In [68]:
t_merged.loc[t_merged['Cluster Labels'] == 0]

,Neighbourhood,Hotel,Cluster Labels,Latitude,Longitude
68,"Rouge Hill, Port Union, Highland Creek",0.025641,0,43.784535,-79.160497
15,Church and Wellesley,0.020000,0,43.665860,-79.383160
78,"Summerhill West, Rathnelly, South Hill, Forest...",0.030000,0,43.686412,-79.400049
13,Central Bay Street,0.030000,0,43.657952,-79.387383
31,"Garden District, Ryerson",0.030000,0,43.657162,-79.378937
39,"Humberlea, Emery",0.021277,0,43.724766,-79.532242
88,"West Deane Park, Princess Gardens, Martin Grov...",0.020833,0,43.650943,-79.554724
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.030000,0,43.628947,-79.394420
64,"Regent Park, Harbourfront",0.020000,0,43.654260,-79.360636
93,"Willowdale, Willowdale East",0.024390,0,43.770120,-79.408493


# CLUSTER 2

In [69]:
t_merged.loc[t_merged['Cluster Labels'] == 1]

,Neighbourhood,Hotel,Cluster Labels,Latitude,Longitude
63,"Queen's Park, Ontario Provincial Government",0.010000,1,43.662301,-79.389494
61,"Parkview Hill, Woodbine Gardens",0.000000,1,43.706397,-79.309937
66,Rosedale,0.010000,1,43.679563,-79.377529
62,Parkwoods,0.010989,1,43.753259,-79.329656
0,Agincourt,0.000000,1,43.794200,-79.262029
59,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.000000,1,43.636258,-79.498509
67,Roselawn,0.000000,1,43.711695,-79.416936
56,"North Toronto West, Lawrence Park",0.000000,1,43.715383,-79.405678
55,"North Park, Maple Leaf Park, Upwood Park",0.000000,1,43.713756,-79.490074
54,"New Toronto, Mimico South, Humber Bay Shores",0.000000,1,43.605647,-79.501321


# CLUSTER 3

In [70]:
t_merged.loc[t_merged['Cluster Labels'] == 2]

,Neighbourhood,Hotel,Cluster Labels,Latitude,Longitude
84,"Toronto Dominion Centre, Design Exchange",0.060000,2,43.647177,-79.381576
18,"Commerce Court, Victoria Hotel",0.060000,2,43.648198,-79.379817
11,Canada Post Gateway Processing Centre,0.080000,2,43.636966,-79.615819
29,"First Canadian Place, Underground city",0.060000,2,43.648429,-79.382280
57,"Northwest, West Humber - Clairville",0.105263,2,43.706748,-79.594054
65,"Richmond, Adelaide, King",0.060000,2,43.650571,-79.384568
5,Berczy Park,0.060000,2,43.644771,-79.373306
76,Stn A PO Boxes,0.060000,2,43.646435,-79.374846
73,St. James Town,0.060000,2,43.651494,-79.375418
35,"Harbourfront East, Union Station, Toronto Islands",0.050000,2,43.640816,-79.381752


# OBSERVATION

### We can see here that Cluster 3 has a large number of hotels, Cluster 1 having a moderate number while Cluster 2 has a very small amount of hotels.

### It seems that Cluster 3 has good competition among hotels and also slightly overcrowded and Cluster 1 has some competition.

### So it is a good opportunity for property developers to build hotels in Cluster 1 which is slowly gaining more competition and might be a great location in the future 

### Or Cluster 2 which has very low hotel ratio and might be a better profitable option

### Hence this project is aimed at project developers who are planning to fina  a great location to build a hotel 

### This project clearly shows that Cluster 1 or Cluster 2 are better options for building a hotel in Toronto

## That's the end of this project
## Thank You